# Data Preprocessing

In this notebook, we will perform different sorts of feature engineering. We will deal
    1. THE MISSING VALUES: 
    a. Categorical Missing Data: Replace with "Empty"
    b. Numerical Missing Data: Replace with the median value since there are outliers
    2. Temporal Features:
    Replace years with number of years from now
    3. Numerical Variable Transformation: 
    From the data analysis section, we found out some numerical variables are skewed. We will perform log normal distribution. Usually performed in continuous variables after bar plotting.
    4. Rare Categorical Feature: 
    If suppose one class among the categorical classes inside a certain feature is less than 1% of the other categorical classes, we omit that class.
    5. Categorial Features with name: Convert into numbers 1, 2, 3 etc in order
    6. Feature Scaling

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split


#### Before the preprocessing, always do the train test split to avoid "Data Leakage"

In [19]:
dataset = pd.read_csv("train.csv")
dataset.head()
X_train,X_test,y_train,y_test=train_test_split(dataset,dataset['SalePrice'],test_size=0.1,random_state=0)

## Missing Data

### Categorical Missing Values

In [20]:
missing_feature = [feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtype == 'O']

def replace_missing_categorical_data(dataset, missing_feature):
    data = dataset.copy()
    data[missing_feature] = data[missing_feature].fillna("Empty")
    return data
dataset = replace_missing_categorical_data(dataset, missing_feature)
dataset[missing_feature].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

### Numerical Missing Values

In [21]:
numerical_missing_feature = [feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtype !="O"]
for feature in numerical_missing_feature:
    median = dataset[feature].median()
    dataset[feature+'nan']=np.where(dataset[feature].isnull(),1,0) #new feature to capture nan values
    dataset[feature].fillna(median,inplace=True)
    
dataset[numerical_missing_feature].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [22]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,60,RL,65.0,8450,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,2,2008,WD,Normal,208500,0,0,0
1,2,20,RL,80.0,9600,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,5,2007,WD,Normal,181500,0,0,0
2,3,60,RL,68.0,11250,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,9,2008,WD,Normal,223500,0,0,0
3,4,70,RL,60.0,9550,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,2,2006,WD,Abnorml,140000,0,0,0
4,5,60,RL,84.0,14260,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,12,2008,WD,Normal,250000,0,0,0


### Temporal Variable

In [25]:
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:      
    dataset[feature]=dataset['YrSold']-dataset[feature]

In [26]:
dataset[['YearBuilt','YearRemodAdd','GarageYrBlt']].head()

,YearBuilt,YearRemodAdd,GarageYrBlt
0,5,5,5.0
1,31,31,31.0
2,7,6,7.0
3,91,36,8.0
4,8,8,8.0


### Numerical Variable Transformation

In [29]:
numerical_skewed_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
#We also considered the SalePrice for the log transformation
for feature in numerical_skewed_features:
    dataset[feature]=np.log(dataset[feature])

In [28]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,60,RL,4.174387,9.041922,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,2,2008,WD,Normal,12.247694,0,0,0
1,2,20,RL,4.382027,9.169518,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,5,2007,WD,Normal,12.109011,0,0,0
2,3,60,RL,4.219508,9.328123,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,9,2008,WD,Normal,12.317167,0,0,0
3,4,70,RL,4.094345,9.164296,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,2,2006,WD,Abnorml,11.849398,0,0,0
4,5,60,RL,4.430817,9.565214,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,12,2008,WD,Normal,12.429216,0,0,0


### Rare Categorical Class

In [31]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype=='O']
for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_variable')

In [36]:
dataset.head(10)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,60,RL,1.428968,2.201872,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,2,2008,WD,Normal,2.505338,0,0,0
1,2,20,RL,1.477511,2.215885,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,5,2007,WD,Normal,2.493950,0,0,0
2,3,60,RL,1.439718,2.233034,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,9,2008,WD,Normal,2.510994,0,0,0
3,4,70,RL,1.409607,2.215315,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,2,2006,WD,Abnorml,2.472277,0,0,0
4,5,60,RL,1.488584,2.258133,Pave,Empty,IR1,Lvl,AllPub,...,Empty,0,12,2008,WD,Normal,2.520050,0,0,0
5,6,50,RL,1.491251,2.257064,Pave,Empty,IR1,Lvl,AllPub,...,Shed,700,10,2009,WD,Normal,2.474065,0,0,0
6,7,20,RL,1.462674,2.221235,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,8,2007,WD,Normal,2.536439,0,0,0
7,8,60,RL,1.443172,2.224389,Pave,Empty,IR1,Lvl,AllPub,...,Shed,350,11,2009,WD,Normal,2.501934,1,0,0
8,9,50,RM,1.369104,2.165541,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,4,2008,WD,Abnorml,2.465938,0,0,0
9,10,190,RL,1.364055,2.187391,Pave,Empty,Reg,Lvl,AllPub,...,Empty,0,1,2008,WD,Normal,2.457744,0,0,0


### Convert Categorical Features into Numbers

In [37]:
for feature in categorical_features:
    labels_ordered=dataset.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    dataset[feature]=dataset[feature].map(labels_ordered)

In [38]:
dataset.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,60,3,1.428968,2.201872,1,2,0,1,1,...,2,0,2,2008,2,3,2.505338,0,0,0
1,2,20,3,1.477511,2.215885,1,2,0,1,1,...,2,0,5,2007,2,3,2.493950,0,0,0
2,3,60,3,1.439718,2.233034,1,2,1,1,1,...,2,0,9,2008,2,3,2.510994,0,0,0
3,4,70,3,1.409607,2.215315,1,2,1,1,1,...,2,0,2,2006,2,0,2.472277,0,0,0
4,5,60,3,1.488584,2.258133,1,2,1,1,1,...,2,0,12,2008,2,3,2.520050,0,0,0
5,6,50,3,1.491251,2.257064,1,2,1,1,1,...,1,700,10,2009,2,3,2.474065,0,0,0
6,7,20,3,1.462674,2.221235,1,2,0,1,1,...,2,0,8,2007,2,3,2.536439,0,0,0
7,8,60,3,1.443172,2.224389,1,2,1,1,1,...,1,350,11,2009,2,3,2.501934,1,0,0
8,9,50,1,1.369104,2.165541,1,2,0,1,1,...,2,0,4,2008,2,0,2.465938,0,0,0
9,10,190,3,1.364055,2.187391,1,2,0,1,1,...,2,0,1,2008,2,3,2.457744,0,0,0


### Feature Scaling

In [42]:
feature_scale=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[feature_scale])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [43]:
scaler.transform(dataset[feature_scale])


array([[0.23529412, 0.75      , 0.49689213, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.57331546, ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.75      , 0.51381745, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.29411765, 0.75      , 0.50263957, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.51381745, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.54995643, ..., 0.        , 0.        ,
        0.        ]])

In [44]:
# transform the train and test set, and add on the Id and SalePrice variables
data = pd.concat([dataset[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(dataset[feature_scale]), columns=feature_scale)],
                    axis=1)

In [45]:
data.to_csv('X_train_clean.csv',index=False)
